In [18]:
# Importing Libraries
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from imblearn.combine import SMOTEENN

In [19]:
df = pd.read_csv("tel_churn.csv")

In [20]:
df.head()

,Unnamed: 0,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,0,29.85,29.85,0,1,0,0,1,1,...,0,0,1,0,1,0,0,0,0,0
1,1,0,56.95,1889.50,0,0,1,1,0,1,...,0,0,0,1,0,0,1,0,0,0
2,2,0,53.85,108.15,1,0,1,1,0,1,...,0,0,0,1,1,0,0,0,0,0
3,3,0,42.30,1840.75,0,0,1,1,0,1,...,1,0,0,0,0,0,0,1,0,0
4,4,0,70.70,151.65,1,1,0,1,0,1,...,0,0,1,0,1,0,0,0,0,0


In [21]:
df = df.drop('Unnamed: 0', axis=1)

x = df.drop('Churn', axis=1)
y = df['Churn']

TRAIN TEST SPLIT

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

DECISION TREE CLASSIFIER

In [25]:
model_dt = DecisionTreeClassifier(criterion="gini", random_state=100, max_depth=6, min_samples_leaf=8)


In [26]:
model_dt.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [27]:
y_pred=model_dt.predict(x_test)
y_pred

array([0, 0, 0, ..., 1, 0, 1])

In [28]:
model_dt.score(x_test,y_test)

0.7967306325515281

In [29]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.83      0.90      0.87      1027
           1       0.66      0.51      0.58       380

    accuracy                           0.80      1407
   macro avg       0.75      0.71      0.72      1407
weighted avg       0.79      0.80      0.79      1407



As you can see that the accuracy is quite low, and as it's an imbalanced dataset, we shouldn't consider Accuracy as our metrics to measure the model, as Accuracy is cursed in imbalanced datasets.

Hence, we need to check recall, precision & f1 score for the minority class, and it's quite evident that the precision, recall & f1 score is too low for Class 1, i.e. churned customers.

Hence, moving ahead to call SMOTEENN (UpSampling + ENN)

In [31]:
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_resample(x, y)

In [32]:
xr_train,xr_test,yr_train,yr_test=train_test_split(X_resampled, y_resampled,test_size=0.2)

In [33]:
model_dt_smote=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=6, min_samples_leaf=8)

In [34]:
model_dt_smote.fit(xr_train,yr_train)
yr_predict = model_dt_smote.predict(xr_test)
model_score_r = model_dt_smote.score(xr_test, yr_test)
print(model_score_r)
print(metrics.classification_report(yr_test, yr_predict))

0.9250645994832042
              precision    recall  f1-score   support

           0       0.93      0.90      0.92       522
           1       0.92      0.95      0.93       639

    accuracy                           0.93      1161
   macro avg       0.93      0.92      0.92      1161
weighted avg       0.93      0.93      0.92      1161



In [35]:
print(metrics.confusion_matrix(yr_test, yr_predict))

[[469  53]
 [ 34 605]]


Random forest classifier

In [36]:
from sklearn.ensemble import RandomForestClassifier

In [37]:
model_rf=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [38]:
model_rf.fit(x_train,y_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [39]:
y_pred=model_rf.predict(x_test)

In [40]:
model_rf.score(x_test,y_test)

0.7995735607675906

In [41]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.82      0.94      0.87      1027
           1       0.71      0.43      0.54       380

    accuracy                           0.80      1407
   macro avg       0.77      0.68      0.70      1407
weighted avg       0.79      0.80      0.78      1407



In [43]:
sm = SMOTEENN()
X_resampled1, y_resampled1 = sm.fit_resample(x,y)

In [44]:
xr_train1,xr_test1,yr_train1,yr_test1=train_test_split(X_resampled1, y_resampled1,test_size=0.2)

In [45]:
model_rf_smote=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [46]:
model_rf_smote.fit(xr_train1,yr_train1)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [47]:
yr_predict1 = model_rf_smote.predict(xr_test1)

In [48]:
model_score_r1 = model_rf_smote.score(xr_test1, yr_test1)

In [49]:
print(model_score_r1)
print(metrics.classification_report(yr_test1, yr_predict1))

0.9370212765957446
              precision    recall  f1-score   support

           0       0.96      0.90      0.93       539
           1       0.92      0.97      0.94       636

    accuracy                           0.94      1175
   macro avg       0.94      0.93      0.94      1175
weighted avg       0.94      0.94      0.94      1175



In [50]:
print(metrics.confusion_matrix(yr_test1, yr_predict1))

[[483  56]
 [ 18 618]]
